In [1]:
import pandas as pd
df_train = pd.read_csv('train.csv', encoding='utf-8')
df_test = pd.read_csv('test.csv', encoding='utf-8')

In [2]:
import re
def preprocessor(text):
    text = re.sub('[\W]+', ' ', text.lower())
    return text

In [3]:
df_train['text'] = df_train['text'].apply(preprocessor)

In [4]:
from sklearn.preprocessing import LabelEncoder
class_le = LabelEncoder()
df_train['author'] = class_le.fit_transform(df_train['author'].values)

In [5]:
from sklearn.model_selection import train_test_split
X, y = df_train.iloc[:, df_train.columns.get_loc('text')].values, \
       df_train.iloc[:, df_train.columns.get_loc('author')].values
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size=0.3,
                                                   random_state=42,
                                                   stratify=y)

In [6]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords as sw
stopwords = sw.words('english')

[nltk_data] Downloading package stopwords to /home/luce/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/luce/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
def tokenizer(text):
    return text.split()

In [8]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [9]:
from nltk.stem import WordNetLemmatizer
lemma = WordNetLemmatizer()
def tokenizer_lemma(text):
    return [lemma.lemmatize(word) for word in text.split()]

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(strip_accents=None,
                       lowercase=False,
                       preprocessor=None)

param_grid = {'vect__ngram_range': [(2,2)],
              'vect__stop_words': [stopwords, None],
              'vect__tokenizer': [tokenizer,
                                  tokenizer_porter,
                                  tokenizer_lemma],
              'clf__penalty': ['l1', 'l2'],
              'clf__C': [0.0025]}

pipe = Pipeline([
                 ('vect', count),
                 ('clf', LogisticRegression(random_state=42))
                ])

model = GridSearchCV(pipe, param_grid, scoring='neg_log_loss', cv=10, verbose=1, n_jobs=-1)
model.fit(X_train, y_train)

Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:  3.8min finished


GridSearchCV(cv=10, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri...alty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'vect__ngram_range': [(2, 2)], 'clf__C': [0.0025], 'vect__stop_words': [['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'he...6378>, <function tokenizer_porter at 0x7fdff8dd6620>, <function tokenizer_lemma at 0x7fdff8dd68c8>]},
       pre_dispatch='2*n_jobs', refit=True, return_t

In [11]:
print(model.best_estimator_)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(2, 2), preprocessor=None, stop_words=None,
        stri...alty='l2', random_state=42,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False))])


In [12]:
print(model.best_params_)

{'vect__ngram_range': (2, 2), 'clf__C': 0.0025, 'vect__stop_words': None, 'clf__penalty': 'l2', 'vect__tokenizer': <function tokenizer_porter at 0x7fdff8dd6620>}


In [13]:
print(model.best_score_)

-1.032428103861736


In [14]:
y_pred_train = model.best_estimator_.predict(X_train)

In [15]:
from sklearn.metrics import confusion_matrix
conf_matrix_train = confusion_matrix(y_true=y_train, y_pred=y_pred_train)
print(conf_matrix_train)

[[5409   23   98]
 [3141  630  173]
 [2808   43 1380]]


In [16]:
from sklearn.metrics import accuracy_score
accuracy_score(y_train, y_pred_train)

0.5413352790952207

In [17]:
y_pred_test = model.best_estimator_.predict(X_test)

In [18]:
conf_matrix_test = confusion_matrix(y_true=y_test, y_pred=y_pred_test)
print(conf_matrix_test)

[[2282   15   73]
 [1446  162   83]
 [1347   36  430]]


In [19]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred_test)

0.48927477017364657

In [20]:
import pickle
import os
pickle.dump(model.best_estimator_,
           open(os.path.join('./pkl_objects', 'bag_2gram_lr_pipe.pkl'), 'wb'),
           protocol=4)